In [ ]:
#ch19_001_20171009.ipynb

In [1]:
from pyspark.sql import SparkSession

In [2]:
#Create SparkSession "spark01"
spark01 = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [3]:
df01 = spark01.read.json("people.json")

In [4]:
df01.show()

+----+-------+------+
| age|   name|status|
+----+-------+------+
|null|Michael|  null|
|  30|   Andy|  null|
|  19| Justin|  null|
|  38|   Jack|     1|
|  32|  Angel|     0|
|  18|Sabrina|     1|
+----+-------+------+



In [5]:
df02=spark.read.csv("test99.data")

In [6]:
df02.show()

+---+---+---+
|_c0|_c1|_c2|
+---+---+---+
|  1|  1|5.0|
|  1|  2|1.0|
|  1|  3|5.0|
|  1|  4|1.0|
|  2|  1|5.0|
|  2|  2|1.0|
|  2|  3|5.0|
|  2|  4|1.0|
|  3|  1|1.0|
|  3|  2|5.0|
|  3|  3|1.0|
|  3|  4|5.0|
|  4|  1|1.0|
|  4|  2|5.0|
|  4|  3|1.0|
|  4|  4|5.0|
+---+---+---+



In [7]:
#spark01.stop()

In [7]:
# Print the schema in a tree format
df01.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)
 |-- status: long (nullable = true)



In [8]:
# Select only the "name" column
df01.select("name").show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
|   Jack|
|  Angel|
|Sabrina|
+-------+



In [10]:
df01.select(df01.name,df01.age+1).show()

+-------+---------+
|   name|(age + 1)|
+-------+---------+
|Michael|     null|
|   Andy|       31|
| Justin|       20|
|   Jack|       39|
|  Angel|       33|
|Sabrina|       19|
+-------+---------+



In [11]:
df01.select(df01["name"],df01["status"]).show()

+-------+------+
|   name|status|
+-------+------+
|Michael|  null|
|   Andy|  null|
| Justin|  null|
|   Jack|     1|
|  Angel|     0|
|Sabrina|     1|
+-------+------+



In [12]:
df02.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)



In [ ]:
# Register the DataFrame as a SQL temporary view

In [25]:
df01.createOrReplaceTempView("people") #將DataFrame df01 登錄為 TempView "people", Spark SQL 可叫用

In [17]:
df02.createTempView("testData") #將DataFrame df02 登錄為 TempView "testData", Spark SQL 可叫用

In [18]:
spark01.sql("SELECT * FROM testData").show()

+---+---+---+
|_c0|_c1|_c2|
+---+---+---+
|  1|  1|5.0|
|  1|  2|1.0|
|  1|  3|5.0|
|  1|  4|1.0|
|  2|  1|5.0|
|  2|  2|1.0|
|  2|  3|5.0|
|  2|  4|1.0|
|  3|  1|1.0|
|  3|  2|5.0|
|  3|  3|1.0|
|  3|  4|5.0|
|  4|  1|1.0|
|  4|  2|5.0|
|  4|  3|1.0|
|  4|  4|5.0|
+---+---+---+



In [22]:
spark01.sql("select _c0 from testData").show()

+---+
|_c0|
+---+
|  1|
|  1|
|  1|
|  1|
|  2|
|  2|
|  2|
|  2|
|  3|
|  3|
|  3|
|  3|
|  4|
|  4|
|  4|
|  4|
+---+



In [24]:
# Select people older than 21, DataFrame API, DataFrame.filter() 
df01.filter(df01['age'] > 21).show()

+---+-----+------+
|age| name|status|
+---+-----+------+
| 30| Andy|  null|
| 38| Jack|     1|
| 32|Angel|     0|
+---+-----+------+



In [26]:
# Select people older than 21, Spark SQL
spark01.sql("select * from people where age > 21").show()

+---+-----+------+
|age| name|status|
+---+-----+------+
| 30| Andy|  null|
| 38| Jack|     1|
| 32|Angel|     0|
+---+-----+------+



In [28]:
# Count people by status, DataFrame API, DataFrame.groupBy()
df01.groupBy("status").count().show()

+------+-----+
|status|count|
+------+-----+
|     0|    1|
|  null|    3|
|     1|    2|
+------+-----+



In [35]:
#Count people by status, Spark SQL
spark01.sql("select status, count(*) as count from people group by status").show()

+------+-----+
|status|count|
+------+-----+
|     0|    1|
|  null|    3|
|     1|    2|
+------+-----+



In [37]:
#SparkSession.sql("SQL Scipt") 回傳的是一個 DataFrame
df03=spark01.sql("select status, count(*) as count from people group by status")

In [38]:
df03.show()

+------+-----+
|status|count|
+------+-----+
|     0|    1|
|  null|    3|
|     1|    2|
+------+-----+



In [ ]:
#Global Temporary View, Spark 2.2 
# Register the DataFrame as a global temporary view
df01.createGlobalTempView("people01")

In [39]:
#========================================================
# Interoperating with RDDs
#========================================================

In [40]:
#Inferring the Schema Using Reflection

In [41]:
from pyspark.sql import Row

In [42]:
sc01=spark01.sparkContext #SparkContext Object "sc01"

In [43]:
type(sc01)

pyspark.context.SparkContext

In [44]:
# Load a text file and convert each line to a Row.

In [48]:
lines01=sc01.textFile("people.txt") #RDD "lines01"

In [49]:
parts01=lines01.map(lambda x:x.split(",")) #RDD.map() --> PipelinedRDD "parts01"

In [50]:
type(parts01)

pyspark.rdd.PipelinedRDD

In [51]:
#叫用 Row function()
people01=parts01.map(lambda x:Row(name=x[0], age=int(x[1]))) #PiplelinedRDD.map() -->RDD "people01"

In [52]:
people01.collect() #RDD "people01" with elements of Row.

[Row(age=29, name=u'Michael'),
 Row(age=30, name=u'Andy'),
 Row(age=19, name=u'Justin'),
 Row(age=38, name=u'Jack'),
 Row(age=32, name=u'Angel'),
 Row(age=18, name=u'Sabrina')]

In [53]:
# Infer the schema, and register the DataFrame as a table.

In [55]:
#使用 SparkSession.createDataFrame(RDD) 將RDD "people01" 轉成DataFrame "dfPeople01
dfPeople01=spark01.createDataFrame(people01) 

In [56]:
dfPeople01.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [57]:
dfPeople01.first()

Row(age=29, name=u'Michael')

In [58]:
dfPeople01.select("name").show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
|   Jack|
|  Angel|
|Sabrina|
+-------+



In [59]:
#登錄一個 TempView "People01"
dfPeople01.createTempView("People01")

In [60]:
#使用 Spark SQL
spark01.sql("select * from People01").show()

+---+-------+
|age|   name|
+---+-------+
| 29|Michael|
| 30|   Andy|
| 19| Justin|
| 38|   Jack|
| 32|  Angel|
| 18|Sabrina|
+---+-------+



In [61]:
# SQL can be run over DataFrames that have been registered as a table.
# The results of SQL queries are Dataframe objects.
teenagersDF = spark01.sql("SELECT name FROM People01 WHERE age >= 13 AND age <= 19")

In [62]:
teenagersDF.show()

+-------+
|   name|
+-------+
| Justin|
|Sabrina|
+-------+



In [73]:
#將DataFrame 轉為 RDD, DataFrame.rdd
teenagersRDD=teenagersDF.rdd.map(lambda x: "Name: " + x.name)

In [75]:
teenagersRDD.collect()

[u'Name: Justin', u'Name: Sabrina']